In [30]:
from utils import *
import importlib
import utils
importlib.reload(utils)
import os.path
from os import path
import pandas as pd
import utils_optimization as opt
importlib.reload(opt)
import time
from scipy.optimize import minimize,NonlinearConstraint,leastsq,fmin_slsqp,least_squares
import numpy.linalg as LA

In [33]:
data_path = pathlib.Path().absolute().joinpath('../2D-3D May 2021')
file_name = data_path.joinpath('record_p2c3_00001_track_outputs_corrected_3D.csv')
df = utils.read_data(file_name,0)
# df = df.rename(columns={'bbrx': 'bblx', 'bblx': 'bbrx', 'fblx': 'fbrx', 'fbrx': 'fblx',
#                        'bbry': 'bbly', 'bbly': 'bbry', 'fbly': 'fbry', 'fbry': 'fbly'})
# df['Timestamp'] = df['Frame #']/30
path_vpts = data_path.joinpath('camera_calibration_points_I24_validation_system.csv')
df = utils.calc_rr_coords(df, 'P2C3', path_vpts) # P1C4 good
A = [36.004654, -86.609976] # south west side, so that x, y coords obey counterclockwise
B = [36.002495, -86.607566]
df = utils.calc_xy(df,A,B)
print(len(df['ID'].unique()))
df = utils.get_x_direction(df)
df = df[df['direction']=='-']
print(len(df['ID'].unique()))
print(df['ID'].unique())
df.head()

76
4
[16231 45231 50231 98231]


,Frame #,Timestamp,ID,Object class,BBox xmin,BBox ymin,BBox xmax,BBox ymax,vel_x,vel_y,...,bbllon,fbr_y,fbl_y,bbr_y,bbl_y,fbr_x,fbl_x,bbr_x,bbl_x,direction
647,426,1.756875e+09,16231,truck (other),152.404480,14.347973,479.062500,112.671280,0.000000,0.000000,...,-86.608027,26.696589,25.732074,26.609000,25.777887,228.947083,229.071121,232.669683,233.092187,-
650,427,1.756875e+09,16231,truck (other),154.751953,-6.557985,475.427551,108.611557,2.347473,-20.905918,...,-86.608058,25.792730,26.974693,25.746963,26.743515,231.940874,231.975470,227.930184,227.809163,-
653,428,1.756875e+09,16231,truck (other),137.678192,-24.991869,467.136475,108.592545,-9.154081,-19.646717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
1490,834,1.756875e+09,45231,truck (other),36.721832,19.708866,392.893433,129.647705,0.000000,0.000000,...,-86.608039,26.673824,25.206049,26.778877,25.254864,227.546736,227.543259,231.999386,231.954811,-
1491,835,1.756875e+09,45231,truck (other),57.700424,-2.272938,390.690796,111.389763,20.978544,-21.981749,...,-86.608065,25.267315,26.973637,25.230647,26.961203,229.797864,229.850401,226.490918,226.572513,-


In [36]:
groups = df.groupby('ID')
groupList = list(groups.groups)
nO = len(groupList) # number of cars to test

ID = 50231
Y1 = groups.get_group(ID)

Y1 = Y1.interpolate(method='linear', limit_area='inside')
Y1 = Y1.dropna()
Y1gps = np.array(Y1[['bbrlat','bbrlon','fbrlat','fbrlon','fbllat','fbllon','bbllat','bbllon']])
Y1 = np.array(Y1[['bbr_x','bbr_y', 'fbr_x','fbr_y','fbl_x','fbl_y','bbl_x', 'bbl_y']]) 
print(len(Y1))
AB = np.array([A,B])
lats = AB[:,0]
lngs = AB[:,1]
utils.draw_map_box(Y1gps[20:40],Y1gps[0,0],Y1gps[0,1], 30, lats, lngs)

6

###############################

 Beginning Key Insertion ...

Key Insertion Completed!!


In [39]:
Y1gps

array([[ 36.00323439, -86.60799527,  36.00326109, -86.60802545,
         36.00325458, -86.60803515,  36.00322787, -86.60800467],
       [ 36.00324337, -86.60800356,  36.00326991, -86.60803395,
         36.00326277, -86.60804428,  36.00323619, -86.60801394],
       [ 36.00325212, -86.60801223,  36.00327861, -86.60804259,
         36.00327107, -86.60805351,  36.00324453, -86.60802322],
       [ 36.00326098, -86.60802061,  36.00328748, -86.60805111,
         36.00327938, -86.60806275,  36.00325282, -86.60803244],
       [ 36.00326948, -86.60802956,  36.00329578, -86.60806001,
         36.00328746, -86.60807178,  36.00326112, -86.60804171],
       [ 36.00327771, -86.60803866,  36.00330328, -86.6080662 ,
         36.00329499, -86.60808004,  36.00326937, -86.60805079]])